In [1]:
from __future__ import annotations
import os, uuid
from typing import List
from PIL import Image
import chromadb
from sentence_transformers import SentenceTransformer
import numpy as np

In [ ]:
# ---------------- Config ----------------
PERSIST_DIR = "./chroma_multimodal_local"
COLLECTION_NAME = "pets_local"

DOG_IMAGES = ["images/dog1.png", "images/dog2.png", "images/dog3.png"]
CAT_IMAGES = ["images/cat1.png", "images/cat2.png", "images/cat3.png"]

In [3]:
def load_images(paths: List[str]) -> List[Image.Image]:
  """โหลดรูปจาก path ที่ให้มา"""
  return [Image.open(path).convert("RGB") for path in paths]

def encode_images(model: SentenceTransformer, images: List[Image.Image]) -> np.ndarray:
  """แปลงรูปเป็น embedding แล้ว normalize"""
  embs = model.encode(images, convert_to_numpy=True, batch_size=4, normalize_embeddings=True)
  return embs

def add_images_to_db(collection, images: List[str], labels: List[str], embs: np.ndarray):
    """เพิ่มรูปและ metadata เข้า Chroma"""
    doc_ids = [str(uuid.uuid4()) for _ in images]
    metadatas = [{"label": lbl, "filename": os.path.basename(path)} for lbl, path in zip(labels, images)]
    documents = [f"A {lbl} image ({os.path.basename(path)})" for lbl, path in zip(labels, images)]

    collection.add(
        ids=doc_ids,
        embeddings=embs.tolist(),
        metadatas=metadatas,
        documents=documents
    )
    print(f"Indexed {len(doc_ids)} images into collection '{collection.name}'.")


In [4]:
# Step 1 - เตรียม path ของรูป
all_images = DOG_IMAGES + CAT_IMAGES
labels = ["dog"] * len(DOG_IMAGES) + ["cat"] * len(CAT_IMAGES)

# Step 2 - โหลด CLIP model
model = SentenceTransformer("clip-ViT-B-32")

# Step 3 - โหลดรูปจาก local
pil_images = load_images(all_images)

# Step 4 - embedding
img_embs = encode_images(model, pil_images)

/opt/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# Step 5 - Setup Chroma
# Reset Chroma system to clear any existing instances
# chromadb.api.shared_system_client.SharedSystemClient._identifier_to_system.clear()

# Step 5 - Setup Chroma
client = chromadb.Client(chromadb.config.Settings(persist_directory=PERSIST_DIR))
collection = client.get_or_create_collection(name=COLLECTION_NAME,  metadata={"hnsw:space": "cosine"})

In [7]:
# Step 6 - เพิ่มรูปเข้า DB
add_images_to_db(collection, all_images, labels, img_embs)

Indexed 6 images into collection 'pets_local'.


In [ ]:
# Step 7 - ทดสอบ Query
query = "dog"
q_emb = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)

res = collection.query(
  query_embeddings=q_emb.tolist(),
  n_results=4,
  include=["metadatas", "documents", "distances"]
)

print(f"\n=== Query: {query} ===")
for rank, (meta, doc, dist) in enumerate(zip(res["metadatas"][0], res["documents"][0], res["distances"][0]), start=1):
  print(f"#{rank} -> {meta['filename']} ({meta['label']}) distance={dist:.4f}")
  print("desc:", doc)

[[ 1.01593984e-02  1.35213425e-02 -2.08940227e-02  2.40679290e-02
  -1.65492073e-02  6.92084990e-03 -2.49350281e-03 -5.33917174e-03
  -2.45604347e-02  4.48840633e-02 -4.69319373e-02 -1.24083813e-02
   2.78852461e-03 -5.47724869e-03  5.93722612e-02  3.10193114e-02
   2.89837737e-02 -5.81763871e-03 -3.22792195e-02 -3.56281772e-02
   2.71745697e-02  1.85842551e-02  2.83535141e-02  1.46791497e-02
   5.35461400e-03  7.95629714e-03  1.17180021e-02  4.10964638e-02
  -3.30236112e-03  6.73852442e-03  2.02235449e-02  6.93910383e-03
   7.61142001e-03 -2.24492494e-02 -1.64683443e-02  5.99655975e-03
   9.60392319e-03  3.23869311e-03 -7.21096434e-03  1.35435853e-02
  -3.85905430e-02  3.73511687e-02  3.17606591e-02  6.35470543e-03
   1.34103606e-02  2.97704488e-02 -1.35256210e-02 -8.43852013e-03
  -1.46808252e-02  1.25547433e-02  4.78481129e-03  1.61112100e-03
   1.44703323e-02 -2.08851229e-03  1.47738401e-02 -3.59925553e-02
   1.11700324e-02  1.91931464e-02 -1.32097714e-02 -2.21656505e-02
   1.80406